# Data Import and Cleaning
## Project 3: Subreddit Classification using NLP
### Robbie Davison

## Imports

In [21]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)

import requests
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier

from nltk.tokenize import RegexpTokenizer

pd.set_option('display.max_columns', None)
%config InlineBackend.figure_format = 'retina'

## Get the 100 most recent submissions from the /r/world news and /r/news subreddits as practice

### Make parameter grid with two different sizes

In [7]:
params_onion = {
    'subreddit': 'TheOnion',
    'size': 1000}

params_not_onion = {
    'subreddit': 'nottheonion',
    'size': 1000}

params_us_news = {
    'subreddit': 'USNEWS',
    'size': 1000} 

params_world_news = {
    'subreddit': 'worldnews',
    'size': 1000}

params_news = {
    'subreddit': 'news',
    'size': 1000}

params_breitbart = {
    'subreddit': 'BreitbartNews',
    'size': 1000}

params_onion = {
    'subreddit': 'TheOnion',
    'size': 1000}
params_onion_s2 = {
    'subreddit': 'TheOnion',
    'size': 10_000}

params_not_onion = {
    'subreddit': 'nottheonion',
    'size': 1000}
params_not_onion_s2 = {
    'subreddit': 'nottheonion',
    'size': 10_000}

params_us_news = {
    'subreddit': 'USNEWS',
    'size': 1000} 
params_us_news_s2 = {
    'subreddit': 'USNEWS',
    'size': 10_000}

params_world_news = {
    'subreddit': 'worldnews',
    'size': 1000}
params_world_news_s2 = {
    'subreddit': 'worldnews',
    'size': 10_000}

params_news = {
    'subreddit': 'news',
    'size': 1000}
params_news_s2 = {
    'subreddit': 'news',
    'size': 10_000}

params_breitbart = {
    'subreddit': 'BreitbartNews',
    'size': 1000}
params_breitbart_s2 = {
    'subreddit': 'BreitbartNews',
    'size': 10_000}

### Make request fetches

In [8]:
base_url = 'https://api.pushshift.io/reddit/submission/search/?'

In [9]:
res_onion = requests.get(base_url, params_onion)

res_not_onion = requests.get(base_url, params_not_onion)

res_us_news = requests.get(base_url, params_us_news)

res_world_news = requests.get(base_url, params_world_news)

res_news = requests.get(base_url, params_news)

res_breitbart = requests.get(base_url, params_breitbart)

print(res_onion, res_not_onion, res_us_news, res_world_news, res_news, res_breitbart)

<Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]>
<Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]> <Response [200]>


res_onion = requests.get(base_url, params_onion)
res_onion_s2 = requests.get(base_url, params_onion_s2)

res_not_onion = requests.get(base_url, params_not_onion)
res_not_onion_s2 = requests.get(base_url, params_not_onion_s2)

res_us_news = requests.get(base_url, params_us_news)
res_us_news_s2 = requests.get(base_url, params_us_news_s2)

res_world_news = requests.get(base_url, params_world_news)
res_world_news_s2 = requests.get(base_url, params_world_news_s2)

res_news = requests.get(base_url, params_news)
res_news_s2 = requests.get(base_url, params_news_s2)

res_breitbart = requests.get(base_url, params_breitbart)
res_breitbart_s2 = requests.get(base_url, params_breitbart_s2)

print(res_onion, res_not_onion, res_us_news, res_world_news, res_news, res_breitbart)
print(res_onion_s2, res_not_onion_s2, res_us_news_s2, res_world_news_s2, res_news_s2, res_breitbart_s2)

### Store all news titles in dataframes

In [16]:
df_onion = pd.DataFrame(res_onion.json()['data'])
df_onion['tag'] = 'onion'

df_not_onion = pd.DataFrame(res_not_onion.json()['data'])
df_not_onion['tag'] = 'not onion'

df_us_news = pd.DataFrame(res_us_news.json()['data'])
df_us_news['tag'] = 'us news'

df_world_news = pd.DataFrame(res_world_news.json()['data'])
df_world_news['tag'] = 'world news'

df_news = pd.DataFrame(res_news.json()['data'])
df_news['tag'] = 'news'

df_breitbart = pd.DataFrame(res_breitbart.json()['data'])
df_breitbart['tag'] = 'breitbart'

df_onion = pd.DataFrame(res_onion.json()['data'])
df_onion['tag'] = 'onion'
df_onion_s2 = pd.DataFrame(res_onion_s2.json()['data'])
df_onion_s2['tag'] = 'onion'

df_not_onion = pd.DataFrame(res_not_onion.json()['data'])
df_not_onion['tag'] = 'not onion'
df_not_onion_s2 = pd.DataFrame(res_not_onion_s2.json()['data'])
df_not_onion_s2['tag'] = 'not onion'

df_us_news = pd.DataFrame(res_us_news.json()['data'])
df_us_news['tag'] = 'us news'
df_us_news_s2 = pd.DataFrame(res_us_news_s2.json()['data'])
df_us_news_s2['tag'] = 'us news'

df_world_news = pd.DataFrame(res_world_news.json()['data'])
df_world_news['tag'] = 'world news'
df_world_news_s2 = pd.DataFrame(res_world_news_s2.json()['data'])
df_world_news_s2['tag'] = 'world news'

df_news = pd.DataFrame(res_news.json()['data'])
df_news['tag'] = 'news'
df_news_s2 = pd.DataFrame(res_news_s2.json()['data'])
df_news_s2['tag'] = 'news'

df_breitbart = pd.DataFrame(res_breitbart.json()['data'])
df_breitbart['tag'] = 'breitbart'
df_breitbart_s2 = pd.DataFrame(res_breitbart_s2.json()['data'])
df_breitbart_s2['tag'] = 'breitbart'

#### Concatenate all dataframes into one master

In [13]:
master = pd.concat([df_onion[['title','tag', 'score']], 
                    df_not_onion[['title','tag', 'score']], 
                    df_us_news[['title','tag', 'score']],
                    df_world_news[['title','tag', 'score']],
                    df_news[['title','tag', 'score']],
                    df_breitbart[['title','tag', 'score']]])

master['y'] = np.where(master['tag'] == 'onion', 1, 0)

master = pd.concat([df_onion[['title','tag', 'score']], 
                    df_not_onion[['title','tag', 'score']], 
                    df_us_news[['title','tag', 'score']],
                    df_world_news[['title','tag', 'score']],
                    df_news[['title','tag', 'score']],
                    df_breitbart[['title','tag', 'score']]])

master_s2 = pd.concat([df_onion_s2[['title','tag', 'score']], 
                    df_not_onion_s2[['title','tag', 'score']], 
                    df_us_news_s2[['title','tag', 'score']],
                    df_world_news_s2[['title','tag', 'score']],
                    df_news_s2[['title','tag', 'score']],
                    df_breitbart_s2[['title','tag', 'score']]])

master['y'] = np.where(master['tag'] == 'onion', 1, 0)
master_s2['y'] = np.where(master_s2['tag'] == 'onion', 1, 0)

## Export Master Dataset to the Data Folder

In [22]:
master.shape

(6000, 4)

In [25]:
master.to_csv('./data/master_1_df.csv', index=False)